In [514]:
import importlib
import utils
import transformers
import model
import losses

In [515]:
from model import *
from utils import *

In [516]:
importlib.reload(utils)
importlib.reload(transformers)
importlib.reload(model)
importlib.reload(losses)

<module 'losses' from 'd:\\Documents\\machinelearning\\cgm-paper\\src\\losses.py'>

In [517]:
print("Current working directory: ", os.getcwd())

Current working directory:  d:\Documents\machinelearning\cgm-paper\src


In [518]:
raw_dir = "../data/raw"
tuning_dir = "../data/tuning"

In [519]:
assert os.path.exists(raw_dir), "Raw data directory not found"
assert os.path.exists(tuning_dir), "Tuning data directory not found"

In [520]:
dataset = load_data(raw_dir)

Found 22 files in ../data/raw


train (139415, 47)
test (32912, 47)


In [521]:
val_dataset = load_data(tuning_dir)

Found 2 files in ../data/tuning
train (13630, 43)
test (3002, 43)


In [522]:
for key in dataset.keys():
    val_dataset[key] = reduce_classes(val_dataset[key])
    dataset[key] = reduce_classes(dataset[key])

In [523]:
dataset.get("train")['Time'].isnull().sum()

0

In [524]:
train_data = get_train_dataset(dataset['train'])
test_data = get_any_dataset(dataset['test'], dataset['train'])

      CGM  month  day  hour  minute  weekday  year  cgm_velo  change  \
24  129.0      5   19    13      35        2  2027  0.003333     1.0   
25  127.0      5   19    13      40        2  2027  0.000000     0.0   
26  124.0      5   19    13      45        2  2027 -0.006667    -2.0   
27  123.0      5   19    13      50        2  2027 -0.010000    -3.0   
28  124.0      5   19    13      55        2  2027 -0.003333    -1.0   

    upper_band  lower_band  wCGM(1)     wCGM(2)     wCGM(3)  wCGM(4)  \
24  198.092207   17.324460    129.0  128.333333  127.666667    127.8   
25  198.840303   20.993031    129.0  129.000000  128.500000    127.9   
26  198.288080   26.128586    127.0  128.333333  128.666667    128.4   
27  196.078435   33.004898    124.0  126.000000  127.500000    128.1   
28  192.197842   41.718824    123.0  123.666667  125.333333    126.8   

       wCGM(5)     wCGM(6)  
24  126.866667  129.476190  
25  127.933333  127.095238  
26  127.933333  127.952381  
27  128.066667  12

The tuning data will be merged with the training data and be fitted into the pipeline to be used for the validation data

In [525]:
tuning_data = get_tuning_dataset(val_dataset['train'], dataset['train'])
val_data = get_any_dataset(val_dataset['test'], tuning_data)

In [526]:
trainX, trainY = train_data.drop(["CGM"], axis = 1), train_data["CGM"]
trainX.shape, trainY.shape

((120010, 16), (120010,))

In [527]:
testX, testY = test_data.drop(["CGM"], axis = 1), test_data["CGM"]
testX.shape, testY.shape

((26845, 16), (26845,))

In [528]:
model = HybridModel()
model.fit(trainX, trainY, testX, testY, eval = True, tune = False)

Base: (18.649991909693895, 4.318563639648477, 0.08439736273418723)
Base + Residuals: (18.528335461606318, 4.304455303706419, 0.08365065423590516) Change %: 4.645254111742716
